## Two weights for the output from the each network (image and audio)

In [1]:
import librosa
from scipy.io.wavfile import read as read_wav
import matplotlib.pyplot as plt
import os

from PIL import Image
from skimage import io, transform

from torch.utils.data import Dataset, DataLoader

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.optim import lr_scheduler
from torchvision import datasets, models, transforms

import time
import copy
import numpy as np
import random

In [21]:
class ToTensor(object):
    """Convert ndarrays in sample to Tensors."""

    def __call__(self, audiofeature):
        mfcc = audiofeature
        # swap color axis because
        # numpy image: H x W x C
        # torch image: C X H X W
        mfcc_tensor = torch.from_numpy(mfcc)
        mfcc_tensor = mfcc_tensor.unsqueeze(0)
        #mfcc = mfcc.transpose((2, 0, 1))
        return mfcc_tensor.double()

In [22]:
class ToTensor_LSTM(object):
    """Convert ndarrays in sample to Tensors."""

    def __call__(self, sample):
        mfcc, label = sample['mfcc'], sample['label']

        # swap color axis because
        # numpy image: H x W x C
        # torch image: C X H X W
        mfcc_tensor = torch.from_numpy(mfcc)
        #mfcc_tensor = mfcc_tensor.unsqueeze(0)
        #mfcc = mfcc.transpose((2, 0, 1))
        return {'mfcc': mfcc_tensor.double(),
                'label': torch.tensor(label).double()}

In [60]:
def audiocombination(directory):
    features = []
    labels = []
    filenames_audio = []
    for actor_number in os.listdir(directory):
        for file in sorted(os.listdir(directory+actor_number)):
            # load the wavefiles
            y, _ = librosa.load(directory+actor_number+'/'+file, sr=48000, offset = 0, duration=3)  # the default sample rate for them is 16kHz, but you can also change that
            S = librosa.feature.melspectrogram(y=y, sr=48000, n_mels=128, fmax=8000)
            feature = librosa.feature.mfcc(S=librosa.power_to_db(S))

            # truncate or zero-pad the signal
            feature_new = np.empty((0, 282))
            for i in range(feature.shape[0]):
                temp = np.concatenate([feature[i], np.zeros(282-feature.shape[1])])
                feature_new = np.append(feature_new, [temp], axis = 0)

            number = file.split("-")
            emotion = number[2]
            #if emotion == "01" or emotion == "02":
            if emotion =="01":
                label = 4 #neutral
            if emotion == "02":
                label = 4 #"neutral"
            elif emotion == "03":
                label = 3 #"happy"
            elif emotion == "04":
                label = 5 #"sad"
            elif emotion == "05":
                label = 0 #"angry"
            elif emotion == "06":
                label = 2 #"fearful"
            elif emotion == "07":
                label = 1 #"disgust"
            elif emotion == "08":
                label = 6 #"surprised" 

            features.append(feature_new)
            labels.append(label)
            filenames_audio.append(file)

    return features, labels, filenames_audio

def imagecombination(directory, single):
    features = []
    filenames_image = []
    for actor_number in os.listdir(directory):
        for videoes in sorted(os.listdir(directory + actor_number)):
            # if only use one frame for image network, then randomly draw one from the frames
            if single:
                numberoffile = len([name for name in os.listdir(directory + actor_number + '/' + videoes)])
                #print(numberoffile)
                index = random.randrange(1, numberoffile-1)
                index = str(index*10)
                target_path = directory + actor_number + '/' +videoes + '/' + index + ".jpg"
                image = io.imread(target_path)
                features.append(image)
            else:
                multiple_image = []
                for index in range(1,7):
                    target_path = directory + actor_number + '/' +videoes + '/' + str(index*10) + ".jpg"
                    image = io.imread(target_path)
                    multiple_image.append(image)
                features.append(multiple_image)
            
            filenames_image.append(videoes)

    return features, filenames_image

In [61]:
class AudioImage_dataset(Dataset):
    def __init__(self, image_path, audio_path, mode, single, image_transform, audio_transform):

        self.image_path = image_path
        self.audio_path = audio_path
        
        self.mode = mode
        self.single = single
        self.image_transform = image_transform
        self.audio_transform = audio_transform
        
        ## Notice the path of image and audio for the train and val is different, add mode in the path
        self.audiofeatures, self.labels, self.filenames_audio = audiocombination(self.audio_path+self.mode+'/')
        self.imagefeatures, self.filenames_image = imagecombination(self.image_path+self.mode+'/', self.single)
        
    
    def __len__(self):
        return len(self.labels)
    

    def __getitem__(self, idx):
        
        audiofeature = self.audiofeatures[idx]
        transformed_audio = self.audio_transform(audiofeature)
        imagefeature = self.imagefeatures[idx]
        torch_list = []
        if not self.single:
            for single_image in imagefeature:
                transformed_image = self.image_transform(single_image)
                torch_list.append(transformed_image)
            final_image = torch.stack(torch_list)
        if self.single:
            final_image = self.image_transform(imagefeature)
        label = self.labels[idx]
        filenames_audio = self.filenames_audio[idx]
        filenames_image = self.filenames_image[idx]
        sample = {'mfcc': transformed_audio, 'image': final_image, 'label': torch.tensor(label).double(), 
                  'filenames_audio': filenames_audio, 'filenames_image': filenames_image}
        
        return sample

In [62]:
class ImageAudio_dataloader():
    def __init__(self, BATCH_SIZE, single, num_workers, image_path, audio_path, image_transform, audio_transform):

        self.BATCH_SIZE=BATCH_SIZE
        self.single = single
        self.num_workers=num_workers
        self.image_path=image_path
        self.audio_path=audio_path
        self.image_transform = image_transform
        self.audio_transform = audio_transform
        #self.in_channel = in_channel
        #self.frame_count ={}
        # split the training and testing videos
        #splitter = UCF101_splitter(path=ucf_list,split=ucf_split)
        #self.train_video, self.test_video = splitter.split_video()
    
    def run(self):
        #print("Now in run ")
        #self.load_frame_count()
        #self.get_training_dic()
        #self.val_sample()
        train_loader, dataset_size_train = self.train()
        val_loader, dataset_size_valid = self.validate()
        
        return train_loader, val_loader, dataset_size_train, dataset_size_valid
    
    def train(self):
        #print("Now in train")
        #applying trabsformation on training videos 
        
        training_set = AudioImage_dataset(image_path=self.image_path, audio_path=self.audio_path,
                                          mode='train', single = self.single, 
                                          image_transform = self.image_transform,
                                          audio_transform = self.audio_transform)
        #print('Eligible videos for training :',len(training_set),'videos')
        dataset_size_train = len(training_set)
        
        train_loader = DataLoader(
            dataset=training_set, 
            batch_size=self.BATCH_SIZE,
            shuffle=True,
            num_workers=self.num_workers)
        return train_loader, dataset_size_train

    def validate(self):
        #print("Now in Validate")
        #applying transformation for validation videos 
        validation_set = AudioImage_dataset(image_path=self.image_path,audio_path=self.audio_path,
                                            mode='valid', single = self.single, 
                                            image_transform = self.image_transform,
                                            audio_transform = self.audio_transform)
        dataset_size_valid = len(validation_set)
        #print('Eligible videos for validation:',len(validation_set),'videos')
        val_loader = DataLoader(
            dataset=validation_set, 
            batch_size=self.BATCH_SIZE, 
            shuffle=True,
            num_workers=self.num_workers)
        return val_loader, dataset_size_valid

In [63]:
# Parameter Setting
batch = 16
single = False
image_path = "./Multimodal-Emotion-Recognition/image_data/"
audio_path = "./Multimodal-Emotion-Recognition/audio_data/"

In [64]:
# Load the data
data_loader = ImageAudio_dataloader(BATCH_SIZE=batch, single = single, num_workers=0,
                                image_path=image_path,       
                                audio_path=audio_path,  
                                image_transform = transforms.Compose([
                                    transforms.ToPILImage(),
                                    transforms.RandomHorizontalFlip(),
                                    transforms.ToTensor(),
                                    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                                         std=[0.229, 0.224, 0.225])
                                ]),
                                audio_transform = transforms.Compose([
                                    ToTensor()
                                ]))
train_loader, valid_loader, dataset_size_train, dataset_size_valid = data_loader.run()

'''
appending train-loader and valid loader for training the model
'''
fullloader = {}
fullloader['train'] = train_loader
fullloader['valid'] = valid_loader
dataset_sizes = {'train': dataset_size_train, 'valid': dataset_size_valid}

In [28]:
# # TEST
# for i, sample_batched in enumerate(fullloader['train']):
#     if i == 0:
#         image = sample_batched['image']
#         print(image[:,1].size())
#     if i < 5:
#         print(i, sample_batched['image'].size(),sample_batched['mfcc'].size(),sample_batched['label'], 
#              sample_batched['filenames_image'], sample_batched['filenames_audio'])

In [65]:
device = torch.device("cuda")
device_tensor = torch.device("cpu")

In [66]:
def train_model(model, criterion, optimizer,scheduler, num_epochs, single, batch):
    since = time.time()
    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0
    
    epoch_loss_list = []
    epoch_acc_list = []

    for epoch in range(1, num_epochs+1):
        print('Epoch {}/{}'.format(epoch, num_epochs))
        print('-' * 10)

        for phase in ['train', 'valid']:
            if phase == 'train':
                scheduler.step()
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0
            
            # Iterate over data.
            for sample_batched in fullloader[phase]:
                image_data = sample_batched["image"]
                audio_data = sample_batched["mfcc"]
                labels = sample_batched['label']
                # print("loading the data")
                # spat_data, temp_data, labels = data
                
                image_data = image_data.to(device)
                audio_data = audio_data.to(device)
                labels = labels.long().to(device)

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(image_data, audio_data.float(),single, batch)
                    #print(outputs)
                    _, preds = torch.max(outputs, 1)
                    
                    loss = criterion(outputs, labels)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                #print("labels: ", labels)
                #print("preds: ", preds)
                #print("loss: ", loss)
                running_loss += loss.item() * image_data.size(0)
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]
            
            epoch_loss_list.append(round(epoch_loss,4))
            epoch_acc_list.append(round(epoch_acc.item(),4))

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                phase, epoch_loss, epoch_acc))

            # deep copy the model
            if phase == 'valid' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())
        
        if epoch % 10 == 0:
            model.load_state_dict(best_model_wts)
            PATH = "./model_fusionLSTM_{}single{}.pth".format(single, epoch)
            torch.save(model.state_dict(), PATH)
        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model, epoch_loss_list, epoch_acc_list


In [2]:
# image_model
image_model = models.alexnet(pretrained=True)
num_ftrs = image_model.classifier[6].in_features
image_model.classifier[6] = nn.Sequential(nn.Linear(num_ftrs,7), nn.Softmax(dim = 1))
image_model.load_state_dict(torch.load("./model_image_new50.pth"))
image_model = image_model.cuda()

In [3]:
# audio_model CNN

class Net(nn.Module):            
    def __init__(self):
        super(Net, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(1, 128, kernel_size=5, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Conv2d(128, 128, kernel_size=5, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Conv2d(128, 128, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(128, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
        )
        self.avgpool = nn.AdaptiveAvgPool2d((6, 6))
        self.classifier = nn.Sequential(
            nn.Dropout(),
            nn.Linear(256 * 6 * 6, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(),
            nn.Linear(4096, 4096),
            nn.ReLU(inplace=True),
            nn.Linear(4096, 7),
            nn.Softmax(dim = 1)
        )

    def forward(self, x):
        x = self.features(x)
        x = self.avgpool(x)
        x = x.view(x.size(0), 256 * 6 * 6)
        x = self.classifier(x)
        return x

audio_model = Net()
audio_model.load_state_dict(torch.load("./model_audio_CNN50.pth"))
audio_model = audio_model.cuda()

In [7]:
# audio model LSTM

class LSTM(nn.Module):
    def __init__(self):
        super(LSTM, self).__init__()
        
        self.hidden_unit = 128  # number of hidden units
        
        # input layer
        self.lstm_layer = nn.LSTM(input_size=20,
                                  hidden_size=self.hidden_unit,
                                  num_layers=2,
                                  batch_first=True,
                                  dropout = 0.2,
                                  bidirectional=False)
        
        self.output_linear = nn.Sequential(nn.Linear(self.hidden_unit, 7),nn.Softmax(dim = 1))
        
    # the function for the forward pass of network (i.e. from input to output)
    def forward(self, input):
        # note that LSTM in Pytorch requires input shape as (batch, seq, feature)
        # so we need to reshape the input
        
        input = torch.squeeze(input)
        #print(input.size())
        batch_size = input.size(0)
        mfcc = input.size(1)
        num_frame = input.size(2)
        
        input = input.transpose(1, 2).contiguous()  # (batch, time, freq)
        #print(input.size())
        
        # pass it through layers
        output, (h_n, c_n) = self.lstm_layer(input)  # (batch, time, hidden) # output[:,-1,:] = h_n[-1], they are all the last hidden state
        #print(output[:,-1,:])
        #print(h_n[-1])
        #output = output.contiguous().view(batch_size*num_frame, -1)  # (batch*time, freq)
        #output = self.output_linear(h_n[-1])
        output = self.output_linear(output[:,-1,:])
        
        # reshape back
        #output = output.view(batch_size, num_frame, 7)
        #output = output.transpose(1, 2).contiguous()  # (batch_size, freq, time)
        
        return output
    
# the model and the optimizers

audio_model = LSTM()
audio_model.load_state_dict(torch.load("./model_audio_LSTM50.pth"))
audio_model = audio_model.cuda()

In [8]:
print(audio_model)

LSTM(
  (lstm_layer): LSTM(20, 128, num_layers=2, batch_first=True, dropout=0.2)
  (output_linear): Sequential(
    (0): Linear(in_features=128, out_features=7, bias=True)
    (1): Softmax()
  )
)


In [9]:
'''
Defining a model model which will do convolution fusion of both stream and 3D pooling 
'''
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.image_feature = image_model #feature_size =   Nx512x7x7
        self.audio_feature = audio_model #feature_size = Nx512x7x7
        #self.layer1       = nn.Sequential(nn.Conv3d(1024, 512, 1, stride=1, padding=1, dilation=1,bias=True),
        #                          nn.ReLU(),nn.MaxPool3d(kernel_size=2,stride=2))
        self.fc           = nn.Sequential(nn.Linear(2,1))
        
    def forward(self,image_data,audio_data,single, batch):
        transform = transforms.Compose([transforms.Normalize(mean=[0],std=[10])])
        if not single:
            x1 = torch.zeros(batch, 7, device=torch.device("cuda"))
            for i in range(6):
                temp = self.image_feature(image_data[:,i])
                temp = temp.to(device_tensor)
                temp = torch.squeeze(transform(temp.unsqueeze(0)))
                temp = temp.to(device)
                x1 += temp
            x1 = x1/6
        if single:
            x1 = self.image_feature(image_data)
        
        x2 = self.audio_feature(audio_data)
        
        temp = torch.cat([x1[:,0].view(16,1),x2[:,0].view(16,1)], dim = 1)
        output = self.fc(temp)
        for i in range(1,7):
            temp = torch.cat([x1[:,i].view(16,1),x2[:,i].view(16,1)], dim = 1)
            temp_output = self.fc(temp)
            #print(temp_output)
            output = torch.cat((output,temp_output),dim = 1)

        return output

In [10]:
model = Net().cuda()

In [11]:
print(model)

Net(
  (image_feature): AlexNet(
    (features): Sequential(
      (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
      (1): ReLU(inplace)
      (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
      (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
      (4): ReLU(inplace)
      (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
      (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (7): ReLU(inplace)
      (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (9): ReLU(inplace)
      (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (11): ReLU(inplace)
      (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    )
    (avgpool): AdaptiveAvgPool2d(output_size=(6, 6))
    (classifier): Sequential(
      (0): Dropout(p=0.5)
      (1): Linear(in_features=9216, out_featu

In [72]:
# for name, param in model.named_parameters():
#     print(name,param.requires_grad)

In [73]:
## Lets freeze the first few layers. This is done in two stages 
# Stage-1 Freezing all the layers 
freeze_layers = 1
if freeze_layers:
    for i, param in model.named_parameters():
        param.requires_grad = False


# Stage-2 , Freeze all the layers till "Conv2d_4a_3*3"
ct = []
for name, param in model.named_parameters():
    if "audio_feature.output_linear.0.bias" in ct:
        param.requires_grad = True
    ct.append(name)   

In [74]:
# Create the optimizer if freeze layer before
params_to_update = model.parameters()
print("Params to learn:")
freeze_layers = 1
if freeze_layers:
    params_to_update = []
    for name,param in model.named_parameters():
        if param.requires_grad == True:
            params_to_update.append(param)
            print("\t",name)
else:
    for name,param in model_ft.named_parameters():
            print("\t",name)

Params to learn:
	 fc.0.weight
	 fc.0.bias


In [75]:
criterion = nn.CrossEntropyLoss()


optimizer = optim.SGD(params_to_update, lr=0.001, momentum=0.9)

# Decay LR by a factor of 0.1 every 10 epochs
scheduler = lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1)


model, loss_list, acc_list = train_model(model, criterion, optimizer,scheduler, num_epochs=50, single = single, batch = batch)

Epoch 1/50
----------
train Loss: 1.9390 Acc: 0.4633
valid Loss: 1.9310 Acc: 0.3375

Epoch 2/50
----------
train Loss: 1.8988 Acc: 0.5475
valid Loss: 1.8897 Acc: 0.3375

Epoch 3/50
----------
train Loss: 1.8216 Acc: 0.5408
valid Loss: 1.8273 Acc: 0.3458

Epoch 4/50
----------
train Loss: 1.7198 Acc: 0.5325
valid Loss: 1.7650 Acc: 0.3458

Epoch 5/50
----------
train Loss: 1.5970 Acc: 0.5375
valid Loss: 1.7388 Acc: 0.3458

Epoch 6/50
----------
train Loss: 1.4842 Acc: 0.5292
valid Loss: 1.7719 Acc: 0.3458

Epoch 7/50
----------
train Loss: 1.4274 Acc: 0.5392
valid Loss: 1.8910 Acc: 0.3375

Epoch 8/50
----------
train Loss: 1.3964 Acc: 0.5442
valid Loss: 2.0644 Acc: 0.3500

Epoch 9/50
----------
train Loss: 1.4139 Acc: 0.5508
valid Loss: 2.3049 Acc: 0.3500

Epoch 10/50
----------
train Loss: 1.4532 Acc: 0.5458
valid Loss: 2.5769 Acc: 0.3375

Epoch 11/50
----------
train Loss: 1.4028 Acc: 0.5408
valid Loss: 2.0877 Acc: 0.3417

Epoch 12/50
----------
train Loss: 1.3923 Acc: 0.5383
valid Los

In [ ]:
filepath = "./fusion3_LSTM_acc.txt"
import json
with open(filepath, 'w') as f:
    f.write(json.dumps(acc_list))

In [59]:
filepath = "./fusion3_LSTM_loss.txt"
import json
with open(filepath, 'w') as f:
    f.write(json.dumps(loss_list))

In [25]:
print(model.fc[0].weight)

Parameter containing:
tensor([[24.4127,  3.2912]], device='cuda:0', requires_grad=True)


In [26]:
a = torch.zeros(5, 7, dtype=torch.float)

In [27]:
print(a)

tensor([[0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0.]])
